In [5]:
import pandas as pd

In [6]:
raw_df = pd.read_csv('raw_data.csv')
incremental_df = pd.read_csv('incremental_data.csv')

In [7]:
# %% [Transformation 1: Handle Missing Values]
print("\n=== TRANSFORMATION 1: Handling Missing Values ===")
print("BEFORE:")
print("Missing values in raw_data:", raw_df.isna().sum())
print("\nMissing values in incremental_data:", incremental_df.isna().sum())


=== TRANSFORMATION 1: Handling Missing Values ===
BEFORE:
Missing values in raw_data: order_id          0
customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64

Missing values in incremental_data: order_id         0
customer_name    6
product          0
quantity         4
unit_price       0
order_date       0
region           2
dtype: int64


In [8]:
def handle_missing(df):
    # Why: Customer names are crucial for analysis
    df['customer_name'] = df['customer_name'].fillna('Unknown')
    
    # Why: Region is important for geographical reporting
    df['region'] = df['region'].fillna('Unspecified')
    
    # Why: Assume at least 1 quantity if not specified
    df['quantity'] = df['quantity'].fillna(1)
    
    # Why: Use median product price when missing
    df['unit_price'] = df['unit_price'].fillna(
        df.groupby('product')['unit_price'].transform('median')
    )
    return df

In [9]:
raw_df = handle_missing(raw_df)
incremental_df = handle_missing(incremental_df)

In [10]:
print("\nAFTER:")
print("Missing values in raw_data:", raw_df.isna().sum())


AFTER:
Missing values in raw_data: order_id         0
customer_name    0
product          0
quantity         0
unit_price       0
order_date       1
region           0
dtype: int64


In [11]:
# %% [Transformation 2: Calculate Total Price]
print("\n=== TRANSFORMATION 2: Calculating Total Price ===")
print("BEFORE SAMPLE:")
print(raw_df[['quantity', 'unit_price']].head(3))


=== TRANSFORMATION 2: Calculating Total Price ===
BEFORE SAMPLE:
   quantity  unit_price
0       1.0       500.0
1       1.0       250.0
2       2.0       250.0


In [12]:
def calculate_totals(df):
    # Why: Business needs total order value
    df['total_price'] = df['quantity'] * df['unit_price']
    
    # Why: Flag high-value orders for priority handling
    df['high_value'] = df['total_price'] > 1000
    return df

raw_df = calculate_totals(raw_df)
incremental_df = calculate_totals(incremental_df)

In [13]:
print("\nAFTER SAMPLE:")
print(raw_df[['quantity', 'unit_price', 'total_price', 'high_value']].head(3))


AFTER SAMPLE:
   quantity  unit_price  total_price  high_value
0       1.0       500.0        500.0       False
1       1.0       250.0        250.0       False
2       2.0       250.0        500.0       False


In [14]:
# %% [Transformation 3: Date Standardization]
print("\n=== TRANSFORMATION 3: Date Standardization ===")
print("BEFORE dtypes:")
print(raw_df.dtypes)


=== TRANSFORMATION 3: Date Standardization ===
BEFORE dtypes:
order_id           int64
customer_name     object
product           object
quantity         float64
unit_price       float64
order_date        object
region            object
total_price      float64
high_value          bool
dtype: object


In [15]:
def process_dates(df):
    # Why: Enable time-based analysis
    df['order_date'] = pd.to_datetime(df['order_date'])
    
    # Why: Simplify monthly reporting
    df['order_month'] = df['order_date'].dt.month_name()
    
    # Why: Identify weekend orders
    df['is_weekend'] = df['order_date'].dt.dayofweek > 4
    return df

In [16]:
raw_df = process_dates(raw_df)
incremental_df = process_dates(incremental_df)

In [17]:
print("\nAFTER dtypes:")
print(raw_df.dtypes)


AFTER dtypes:
order_id                  int64
customer_name            object
product                  object
quantity                float64
unit_price              float64
order_date       datetime64[ns]
region                   object
total_price             float64
high_value                 bool
order_month              object
is_weekend                 bool
dtype: object


In [18]:
# %% [Transformation 4: Data Quality]
print("\n=== TRANSFORMATION 4: Data Quality ===")
print(f"BEFORE: {len(raw_df)} rows in raw_data")


=== TRANSFORMATION 4: Data Quality ===
BEFORE: 100 rows in raw_data


In [19]:
def clean_data(df):
    # Why: Remove duplicate orders (business rule: order_ids must be unique)
    df = df.drop_duplicates(subset='order_id', keep='first')
    
    # Why: Remove records with impossible prices
    df = df[df['unit_price'] > 0]
    return df

In [20]:
raw_df = clean_data(raw_df)
incremental_df = clean_data(incremental_df)

In [21]:
print(f"AFTER: {len(raw_df)} rows in raw_data")

AFTER: 99 rows in raw_data
